## 1. import 키워드로 필수 라이브러리 불러오기
#### ※ 라이브러리 : 자주 쓰이는 기능들을 미리 만들어놓고 필요할 때 마다 가져다 쓸 수 있게 만든 것
    - numpy
      - 길어질 수 있는 코드들을 짧게 해결해주는 라이브러리
    - pandas
      - 파이썬에서 엑셀 관련 작업을 할 수 있게 해주는 라이브러리
    - math
      - 수학 관련 함수를 추가해주는 라이브러리 (반올림, sin, cos 등등)
    - xlwings
      - 보안프로그램으로 인해 파이썬이 직접 엑셀파일을 열어 데이터를 불러오게하는 라이브러리
      

In [ ]:
import numpy as np
import pandas as pd
import math
import xlwings as xw

## 2. 엑셀 불러오기
- xw.Book('input.xlsx')
  - input.xlsx 파일을 불러와 book 변수에 저장합니다.
- book.sheets[0]
  - input.xslx의 **0번째 시트**데이터를 sheet에 저장합니다.
- sheet.used_range.options(pd.DataFrame, index=False).value
  - **0번째 시트**데이터를 파이썬에서 작업할 수 있도록 Pandas의 DataFrame으로 변환해주는 역할을 합니다.
- book.close()
  - input.xlsx 파일을 닫아줍니다.

In [ ]:
#### 약 5초 소요
input("[정보] input.xlsx 파일을 불러옵니다. 파일이 준비되어 있으면 Enter키를 눌러주세요.")
book = xw.Book('input.xlsx')
sheet = book.sheets[0]
target = sheet.used_range.options(pd.DataFrame, index=False).value
book.close()
print("[정보] input.xlsx 파일을 불러왔습니다.")

## 3. head(n)을 통해 맨 위에서 n개 만큼 불러오기
현재 pandas 타입의 변수는 target이고, target.head(1)을 하면 상단에 있는 열 1개를 가져와 보여주게 됩니다.

In [ ]:
target.head(1)

## 4. 고객명 + 생일 을 합친 열 추가하기
정상적인 고객명은 '에_윤용권_720403'으로 볼 수 있습니다.


고객명을 '윤용권720403'으로 편집하고 싶으면, target['고객명'].str.split("_")을 이용할 수 있습니다.

In [ ]:
print(target['고객명'].str.split("_"))

[에, 윤용권, 720403]인 리스트로 출력되는걸 확인할 수 있고, 1번 인덱스와 2번 인덱스를 합치면 될 것으로 보입니다.

Pandas에서는 target['열 이름']을 통해 기존에 존재하는 모든 열 데이터를 가져올 수 있습니다.

In [ ]:
target['고객명'].head()

또는 밑 코드처럼 새로운 열을 만드는데에도 활용됩니다.


'고객명생일' 열을 새로 생성하고, 셀 값으로 그 행에 맞는 값을 지정할 수 있습니다.

In [ ]:
#고객명+생일로 열 추가
target['고객명생일'] = target['고객명'].str.split("_").str[1] + target['고객명'].str.split("_").str[2]
target[['주문일자', '고객명','고객명생일', '주문유형']].head(10)

0번과 1번 행이 고객명이 부정확하여 고객명생일이 NaN으로 입력되어 있는데, split에 실패하여 발생한 현상입니다. 보통 고객명에도 업체명이 들어가있거나, 이름만 들어가있는 경우에 NaN이 입력되어 있었습니다.
- None : 처음부터 빈 셀을 의미
- NaN : 계산 과정에서 잘못된 입력을 받았음을 의미


둘 다 분류과정에서는 제외되어야 하기 때문에, 통일성을 위해 아래에서 NaN을 None으로 전부 변경했습니다.

In [ ]:
target = target.replace({np.nan: None})
person_list = target['고객명생일']
target[['주문일자', '고객명','고객명생일', '주문유형']].head(10)

이제 for 반복문을 돌리기 위해 None을 제외한 '고객명생일' 항목을 리스트로 새로 만들어줍니다. 여기서 방금 None(혹은 NaN) 항목은 제외되어야 하므로, 밑 코드에서 None 항목을 제외합니다.

동일인은 1번만 있으면 되기 때문에 중복 항목도 제외하겠습니다.

In [ ]:
# 고객명생일 형식 아닌거 지우기 (재단명, Null값은 여기서 삭제됨)
person_list = list(filter(None,person_list))
print(person_list)

#중복제거
person_list = list(dict.fromkeys(person_list))
print(person_list)

## 5. 분류 시작
* for p in person_list:
  - p에는 고객명 생일이 들어갑니다. (윤용권720403, 권태질360730으로 총 2번 실행)
* for i in target.loc[target['고객명생일'].str.contains(p, na=False, regex=False)]['주문유형'].tolist():
  - #### i에는 해당하는 행의 '주문유형'이 입력되어 있습니다. 
  - p가 '윤용권720403' 일 경우
    - 전체 항목에서 고객명생일이 '윤용권720403'인 행의 주문유형을 모두 가져옵니다.
    - 1가지 항목만 있고, ZZA 항목이기 때문에 value는 1이 됩니다.
    - if value >= 1: 항목에 통과했지만 for i in range(value-1): 에서는 반복할 필요가 없기 때문에 삭제되는것 없이 결과에 추가됩니다.
  - p가 '권태질360730' 일 경우
    - 전체 항목에서 고객명생일이 '권태질360730'인 행의 주문유형을 모두 가져옵니다.
    - 3가지 항목이 있고, ZZA 항목은 2개, ZZB 항목은 1개이기 때문에 value는 1이 됩니다. (+2, -1)
    - if value >= 1:을 통과했습니다.
    - tmp_df = target.loc[target['고객명생일'].str.contains(p, na=False, regex=False) & target['주문유형'].str.contains('YKB2-ZZA', na=False, regex=False),:]
      - 중복된 ZZA 항목 삭제를 위해 고객명생일이 '권태질360730'인 전체 데이터에서 'YKB2-ZZA'인 행들만 가져옵니다.
    - for i in range(value-1):
      - ZZA는 2개이므로, 1번 실행되게 됩니다.
      - tmp_df = tmp_df.drop(tmp_df.index[0])
        - 맨 위에 있는 항목이 제일 오래된 항목이므로, 하나만 남을때까지 상단에 위치한 데이터를 하나씩 삭제합니다.

In [ ]:
print("[정보] 분류를 시작합니다")
i = 0
value = 0
arr_df = []

for p in person_list:
    value = 0
    for i in target.loc[target['고객명생일'].str.contains(p, na=False, regex=False)]['주문유형'].tolist(): 
        if i == 'YKKR-ZFM': #설치 후 고장
            value -= 1
        elif i == 'YKA1-ZZB': #단순변심
            value -= 1
        elif i == 'YKB2-ZZA': #설치계약
            value += 1
        # 1일 경우 일단 설치계약 된걸로 이것만 가져가면 됨!
        # 0과 같거나 보다 작을경우 설치되지 않았으니 걸러야함!
        # 1보다 클 경우 중복주문 된거니 마지막에 주문된걸 살리고 상단에 있는 주문을 걸러야함!
    
    if value >= 1:
        tmp_df = target.loc[target['고객명생일'].str.contains(p, na=False, regex=False) & target['주문유형'].str.contains('YKB2-ZZA', na=False, regex=False),:]
        value = len(tmp_df)
        for i in range(value-1):
            print("[정보]", p, "님의 일부 데이터를 삭제합니다.")
            tmp_df = tmp_df.drop(tmp_df.index[0]) # 맨 위에것만 지우면 가장 최신(마지막)인 맨 밑에만 남을테니 0번 인덱스 삭제
        arr_df.append(tmp_df)

output_target = pd.concat(arr_df)


In [ ]:
output_target = output_target.sort_index()

## 6. 저장
pandas 타입의 객체에서 .to_excel() 함수를 통해 작업이 완료된 판다스 객체를 엑셀파일로 만들어냅니다.   
index=False는 맨 앞 인덱스 번호를 제외하고 파일로 출력하는것을 의미합니다.

In [ ]:
output_target.to_excel("output.xlsx", engine="openpyxl", index=False)
input("[정보] output.xlsx 파일을 확인해주세요. (Enter키를 누르거나 상단의 X를 눌러 닫으실 수 있습니다)")